In [ ]:
import mlflow
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# get mlflow runs
experiment_id = '248049560939767354' # TODO update mlflow experiment ID if it changes (check mlruns directory)

mlflow.set_tracking_uri("../../mlruns")

runs = mlflow.search_runs(experiment_ids=[experiment_id])

failed_runs = len(runs[runs['status']=='FAILED'][['params.model']])
print("{} experiment runs failed ({}% of total)".format(failed_runs, failed_runs/len(runs)*100))

In [ ]:
# timestamp for saving figures, tables and other outputs from this experiment run
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
print(timestamp)

In [ ]:
runs.to_csv(f'results-{timestamp}.csv', index=None)

In [ ]:
# cleanup the data fields for analysis

data = runs[['params.model',
      'params.datafile',
      'params.num_groups',
      'params.inference_type',
      'params.interventional_option',
      'metrics.RMSE_avg_val',
      'metrics.RMSE_avg_test',
      'params.lambda',
      'params.num_initial_epochs',
      'metrics.total_num_epochs',
      'params.base_learning_rate_initial', 
      'params.meta_learning_rate_initial', 
      'params.base_learning_rate_main', 
      'params.meta_learning_rate_global_main',
      'params.meta_learning_rate_group_main']]

data['method'] = data['params.model'] + data['params.inference_type'] + data['params.interventional_option']
data['dataset'] = data['params.datafile'].str.split('/').str[-1].str.split('_').str[0]
data['trials'] = data['params.datafile'].str.split('/').str[-1].str.split('_').str[1].str[7].astype(int)


In [ ]:
# economics dataset  
data = data[~data['metrics.RMSE_avg_test'].isna()]
results = data[data['dataset']=='econ'].sort_values(by='metrics.RMSE_avg_val').groupby(['dataset','method','trials']).first()
results = results.groupby(['dataset','method'])[['metrics.RMSE_avg_val','metrics.RMSE_avg_test']].agg(['mean', 'std']).reset_index()
results['metrics.RMSE_avg_val_fmt'] = results.apply(lambda x: f"{x[('metrics.RMSE_avg_val','mean')]:.3f} ({x[('metrics.RMSE_avg_val','std')]:.3f})", axis=1)
results['metrics.RMSE_avg_test_fmt'] = results.apply(lambda x: f"{x[('metrics.RMSE_avg_test','mean')]:.3f} ({x[('metrics.RMSE_avg_val','std')]:.3f})", axis=1)
results = results.sort_values(by=('metrics.RMSE_avg_test','mean'))[['method','metrics.RMSE_avg_val_fmt','metrics.RMSE_avg_test_fmt']]
results.to_csv(f'results-econ-{timestamp}.csv', index=None)
results

In [ ]:
# ukbiobank dataset
data = data[~data['metrics.RMSE_avg_test'].isna()]
results = data[(data['dataset']=='ukbb')&(data['trials'].isin([1,2]))].sort_values(by='metrics.RMSE_avg_val').groupby(['dataset','method','trials']).first()
results = results.groupby(['dataset','method'])[['metrics.RMSE_avg_val','metrics.RMSE_avg_test']].agg(['mean', 'std']).reset_index()
results['metrics.RMSE_avg_val_fmt'] = results.apply(lambda x: f"{x[('metrics.RMSE_avg_val','mean')]:.3f} ({x[('metrics.RMSE_avg_val','std')]:.3f})", axis=1)
results['metrics.RMSE_avg_test_fmt'] = results.apply(lambda x: f"{x[('metrics.RMSE_avg_test','mean')]:.3f} ({x[('metrics.RMSE_avg_val','std')]:.3f})", axis=1)
results = results.sort_values(by=('metrics.RMSE_avg_test','mean'))[['method','metrics.RMSE_avg_val_fmt','metrics.RMSE_avg_test_fmt']]
results.to_csv(f'results-ukbb-{timestamp}.csv', index=None)
results